![ADSA Logo](http://i.imgur.com/BV0CdHZ.png?2 \"ADSA Logo\")

# ADSA Workshop 3 - Introduction to Databases and MySQL

In this workshop code session, we will write a small program to obtain and visualize weather forecasts for the next 5 days. This program will be done in two parts:
1. In the first part, we will collect 5 day weather forecast data and store that data in a MySQL database.
2. In the second part, we will read data from this table and then create a visualization of the data.

***
## Setting up the database for this workshop

First, log into the `mysql shell` using the following command:

    mysql -u root -p
    
When prompted for the password, use `adsauiuc`. This user and password combination was set on installation.

Next, we shall create a new database for this workshop. Type this:

    CREATE DATABASE workshop3;
    
Do not forget the semi-colon at the end! Next, we will create a new user that will have access to modify this database. The password for this user is specified using `IDENTIFIED BY`.

    CREATE USER 'wkshp_user'@'localhost' IDENTIFIED BY 'sqlrocks';
        
We will grant admin permissions to our user on this database using

    GRANT ALL ON workshop3.* TO 'wkshp_user'@'localhost';
    
And then exit the MySQL shell by typing:

    quit

***
## Connecting to our database

In [ ]:
# import the MySQLdb module and the modified print function.

import MySQLdb as mdb
from __future__ import print_function

In [ ]:
# create the connect object
# parameters are (host, user, password, database)
con = mdb.connect('localhost', 'wkshp_user', 'sqlrocks', 'workshop3');

with con:
    cursor = con.cursor()
    cursor.execute("SELECT VERSION()")

    ver = cursor.fetchone()
    
    print("Database version : {0}".format(ver))

**NOTE**: When using the `with` syntax, Python automagically manages the resources needed to open a connection to the database and close it. Else our code would look like this:

    try:
        con = mdb.connect('localhost', 'testuser', 'test623', 'testdb');
        cur = con.cursor()
        cur.execute("SELECT VERSION()")
        ver = cur.fetchone()
        print("Database version : {0}".format(ver))
    
    except mdb.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        import sys
        sys.exit(1)
    
    finally:
        if con:
            con.close()

***
## Getting forecast data from OpenWeatherMap

We are going to look at the 5 day weather forecast for Champaign (recorded as city ID `4887158` in OpenWeatherMap's data). The API call we are going to use is `api.openweathermap.org/data/2.5/forecast?id={city_id}`. This call returns weather forecast data for 5 days, with a new datapoint every 3 hours.

Here is what a sample data point looks like in the `weather_data['list']` array. The objects we are concerned with here are `main['temp_min']`, `main['temp_max']`, `dt_txt`, `weather[0]['main']`, and `weather[0]['description']` which will give us the data and time of the data point.

    {
        "dt":1442728800,
        "main": {
            "temp":282.11,
            "temp_min":282.11,
            "temp_max":283.319,
            "pressure":1006.96,
            "sea_level":1033.48,
            "grnd_level":1006.96,
            "humidity":88,
            "temp_kf":-1.21
        },
        "weather":[
            {  
                "id":800,
                "main":"Clear",
                "description":"sky is clear",
                "icon":"01n"
            }
        ],
        "clouds":{
            "all":0
        },
        "wind":{
            "speed":1.25,
            "deg":13.0026
        },
        "rain":{
        },
        "sys":{
            "pod":"n"
        },
        "dt_txt":"2015-09-20 06:00:00"
    }

In [ ]:
# So let's parse this data to a Python Dictionary

import urllib2

# Champaign, IL has the city_id 4887158
url = 'http://api.openweathermap.org/data/2.5/forecast?id=4887158'
response = urllib2.urlopen(url)
weather_html = response.read()

from json import JSONDecoder, dumps

decoder = JSONDecoder()
weather_data = decoder.decode(weather_html)

pretty_weather_data = dumps(weather_data, sort_keys=True, indent=2, separators=(',', ': '))
# print(pretty_weather_data)

### Extracting Minimum and Maximum Temperatures `(dt_txt, main['temp_min'], main['temp_max'])`

In [ ]:
# Python dictionary that stores the list data
data_points = weather_data['list']
# An empty list that will store pairs of temperature and date-time as (temp, date-time)
hourly_temp_list = []

for datum in data_points:
    dt = datum['dt_txt']
    temp_min = datum['main']['temp_min']
    temp_max = datum['main']['temp_max']
    hour_temp_tuple = (dt, temp_min, temp_max)
    hourly_temp_list.append(hour_temp_tuple)
    
# Print some sample data
for i in range(10):
    print(hourly_temp_list[i])

We see that for each day, there are data points starting at 12AM GMT until 9PM GMT. We want to convert this data and store it in the database as shown in this schema:

| tempDate | min    | max    |
|----------|--------|--------|
| 20150920 | 261.63 | 276.77 |
| 20140921 | 281.44 | 295.32 |
| ...      | ...    | ...    |

Note the format of the date, we have removed the hyphens to store it as an integer.

Here's a pre-written function that will convert the hourly weather data to a day-average format. You do not need to understand how it works, just compile it and see the transformed results.

In [ ]:
def convert_temp_data_to_schema(data):
    '''
    This function takes in a list of hourly weather data as (date_time, min_temp, max_temp)
    and returns the data as day max and min temps. You DO NOT NEED TO understand how this function works.
    '''
    minmax_temps = []
    
    day = None
    hourCount = 0.0
    min_temp = 0.0
    max_temp = 0.0
    
    for (h_day, h_min, h_max) in data:
        if day is None:
            day = h_day
            min_temp = h_min
            max_temp = h_max
            
        if day[:10] == h_day[:10]:
            hourCount = hourCount + 1.0
            if h_min < min_temp:
                min_temp = h_min
            if h_max > max_temp:
                max_temp = h_max
        else:
            day = day[:10]
            day = day.replace('-', '')
            day = int(day)
            minmax_temp_tuple = (day, min_temp, max_temp)
            
            minmax_temps.append(minmax_temp_tuple)
            
            day = None
            min_temp = 0.0
            max_temp = 0.0
            hourCount = 0.0
            
    return minmax_temps

In [ ]:
minmax_temp_list = convert_temp_data_to_schema(hourly_temp_list)

# print the converted data
print(minmax_temp_list)

In [ ]:
cursor.execute("DROP TABLE IF EXISTS Temps")
cursor.execute("CREATE TABLE Temps(id INT PRIMARY KEY AUTO_INCREMENT, tempDate INT, \
                min DECIMAL(6,3), max DECIMAL(6,3))")

The format for adding a row to our table is:

    cursor.execute("INSERT INTO Temps(tempDate,min,max) VALUES({day},{min_temp},{max_temp})")
    
We will run a for-loop and insert the data into our `Temps` table.

In [ ]:
for (day, min_temp, max_temp) in minmax_temp_list:
    cursor.execute("INSERT INTO Temps(tempDate,min,max) VALUES({0},{1},{2})"
                   .format(day, min_temp, max_temp))

Let's try to query the data and see if it's all properly stored.

In [ ]:
cursor.execute("SELECT * FROM {0}".format("Temps"))
rows = cursor.fetchall()

for row_data in rows:
    print(row_data)

### Extracting Weather Descriptions `(dt_text, weather[0]['main'], weather[0]['description'])`

In [ ]:
# Python dictionary that stores the list data
data_points = weather_data['list']
# An empty list that will store pairs of temperature and date-time as (temp, date-time)
hourly_desc_list = []

for datum in data_points:
    dt = datum['dt_txt']
    weather_main = datum['weather'][0]['main']
    weather_desc = datum['weather'][0]['description']
    hour_desc_tuple = (dt, weather_main, weather_desc)
    hourly_desc_list.append(hour_desc_tuple)
    
# Print some sample data
for i in range(10):
    print(hourly_desc_list[i])

Similar to the previous table, we want to store our description data using a schema that looks like this:

| tempDate | main     | description     |
|----------|----------|-----------------|
| 20150920 | 'Clear'  | 'sky is clear'  |
| 20140921 | 'Cloudy' | 'broken clouds' |
| ...      | ...      | ...             |

In [ ]:
def convert_desc_data_to_schema(data):
    '''
    This function takes in a list of hourly weather data as (date_time, main, desc)
    and returns the data as a per-day description. You DO NOT NEED TO understand how this function works.
    '''
    descriptions = []
    
    day = None
    hourCount = 0.0
    main = 'Clear'
    desc = 'sky is clear'
    
    for (h_day, h_main, h_desc) in data:
        if day is None:
            day = h_day
            main = h_main
            desc = h_desc
            
        if day[:10] == h_day[:10]:
            hourCount = hourCount + 1.0
            if h_main != 'Clear':
                main = h_main
            if h_desc != 'sky is clear':
                desc = h_desc
        else:
            day = day[:10]
            day = day.replace('-', '')
            day = int(day)
            desc_tuple = (day, main, desc)
            
            descriptions.append(desc_tuple)
            
            day = None
            main = 'Clear'
            desc = 'sky is clear'
            hourCount = 0.0
            
    return descriptions

In [ ]:
desc_list = convert_desc_data_to_schema(hourly_desc_list)

# print the converted data
print(desc_list)

Data looks great again. Let's store this data in it's own `Descriptions` table.

In [ ]:
cursor.execute("DROP TABLE IF EXISTS Descs")
cursor.execute("CREATE TABLE Descs (id INT PRIMARY KEY AUTO_INCREMENT, tempDate INT, \
                main VARCHAR(15), description VARCHAR(25))")

We will use a for-loop again to insert our data. Note how VARCHARs have to be enclosed in single-quotes when inserting them into the database.

In [ ]:
for (day, main_desc, description) in desc_list:
    cursor.execute("INSERT INTO Descs(tempDate,main,description) \
                    VALUES({0}, '{1}', '{2}')".format(day, main_desc, description))

Let's query the data again to make sure everything is inserted.

In [ ]:
cursor.execute("SELECT * FROM {0}".format("Descs"))
rows = cursor.fetchall()

for row_data in rows:
    print(row_data)

So all of our data has been stored perfectly. Let's move on to the other program that will query both tables and join the data.

***
## Joining Tables With Related Data

This is what the data in our two tables `Temps` and `Descs` looks like now.

| tempDate | min    | max    |      | tempDate | main     | description     |
|----------|--------|--------|------|----------|----------|-----------------|
| 20150920 | 261.63 | 276.77 |      | 20150920 | 'Clear'  | 'sky is clear'  |
| 20140921 | 281.44 | 295.32 |      | 20140921 | 'Cloudy' | 'broken clouds' |
| ...      | ...    | ...    |      | ...      | ...      | ...             |

You can see how there is related data in the two tables. Both tables have a date column, and are followed by columns that describe the weather on that day. Why don't we combine these two tables?

For this task, we will use a join clause.

Here's what SQL joins look like visually:
![SQL Join Venn Diagram](http://bilquist.com/wp-content/uploads/2014/06/SQL-Join-Venn-Diagrams.jpg)

Since we want to match related data that exist in both tables, we want an intersection of both sets, or in SQL terms, we want to do an inner join.

In [ ]:
# we want the final output as (date, min_temp, max_temp, main_desc, description)

cursor.execute("SELECT {0}.tempDate, {0}.min, {0}.max, {1}.main, {1}.description \
                FROM {0} INNER JOIN {1} \
                ON {0}.tempDate = {1}.tempDate"
                .format("Temps","Descs"))
rows = cursor.fetchall()

plot_data = rows

for row_data in rows:
    print(row_data)

Now that we have this data, let's visualize it!

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

days = []
min_temps = []
max_temps = []

for data in plot_data:
    day = int(data[0]) % 100
    days.append(day)
    
    min_temp_fah = 1.8 * (int(data[1]) - 273.15) + 32
    min_temps.append(min_temp_fah)
    
    max_temp_fah = 1.8 * (int(data[2]) - 273.15) + 32
    max_temps.append(max_temp_fah)

plt.plot(days, min_temps, linewidth=2, label='Minimum Temperatures')
plt.plot(days, max_temps, linewidth=2, label='Maximum Temperatures')
plt.xticks(days)
plt.xlabel('Day')
plt.ylabel('Temperature')
plt.legend(bbox_to_anchor=(1.6, 0.6))
plt.show()